In [2]:
import pandas as pd 
from datetime import datetime
import numpy as np

In [3]:
df = pd.read_csv("../../csv/AMZN.csv", index_col=0)

In [4]:
# 결측치, 양의 무한대, 음의 무한대를 제거 
flag = df.isin( [np.nan, np.inf, -np.inf] ).any(axis=1)
df = df.loc[~flag, ]

In [5]:
# 인덱스가 Date가 아니라면 Date 컬럼을 인덱스로 변경
# 조건식 : 컬럼들 중에 Date가 존재한다면
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

In [6]:
# index를 시계열데이터로 변경 
# pandas에 내장된 to_datetime() 함수 이용
pd.to_datetime(df.index)

DatetimeIndex(['1997-05-15', '1997-05-16', '1997-05-19', '1997-05-20',
               '1997-05-21', '1997-05-22', '1997-05-23', '1997-05-27',
               '1997-05-28', '1997-05-29',
               ...
               '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14',
               '2019-06-17', '2019-06-18', '2019-06-19', '2019-06-20',
               '2019-06-21', '2019-06-24'],
              dtype='datetime64[ns]', name='Date', length=5563, freq=None)

In [7]:
# datetime 라이브러리 안에 있는 strptime() 함수 이용
datetime.strptime( df.index[0], '%Y-%m-%d' )

datetime.datetime(1997, 5, 15, 0, 0)

In [8]:
df.index = df.index.map(
    lambda x : datetime.strptime(x, '%Y-%m-%d')
)

In [9]:
# 투자의 시작 시간과 종료 시간을 통해서 df의 인덱스를 필터링
start = '2010-01-01'
end = '2015-01-01'
t_s = datetime.strptime(start, '%Y-%m-%d')
t_e = datetime.strptime(end, '%Y-%m-%d')

In [10]:
df.loc[t_s : t_e, ]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,136.250000,136.610001,133.139999,133.899994,133.899994,7599900
2010-01-05,133.429993,135.479996,131.809998,134.690002,134.690002,8851900
2010-01-06,134.600006,134.729996,131.649994,132.250000,132.250000,7178800
2010-01-07,132.009995,132.320007,128.800003,130.000000,130.000000,11030200
2010-01-08,130.559998,133.679993,129.029999,133.520004,133.520004,9830500
...,...,...,...,...,...,...
2014-12-24,306.380005,307.000000,302.880005,303.029999,303.029999,1513800
2014-12-26,305.000000,310.779999,303.809998,309.089996,309.089996,2893800
2014-12-29,307.850006,314.269989,306.579987,312.040009,312.040009,3009000


In [11]:
price_df = df.loc[start : end, ['Adj Close'] ]

In [12]:
# 첫날(구매)의 수정종가, 마지막날(판매)의 수정종가
price_df.iloc[-1, 0] / price_df.iloc[0, 0]

2.317774607219176

In [13]:
price_df['rtn'] = \
    (price_df['Adj Close'].pct_change() + 1).fillna(1)

In [14]:
price_df['rtn'].cumprod()[-1]

C:\Users\moons\AppData\Local\Temp\ipykernel_17504\197219032.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price_df['rtn'].cumprod()[-1]


2.317774607219178

### Buy and Hold 함수화
- 매개변수의 개수는 4개?
    - 데이터프레임(_df) -> 필수
    - 투자 시작 시간(매수)(_start) -> '2010-01-01' 기본값 설정
    - 투자 종료 시간(매도)(_end) -> 현재 시간 기본값
    - 특정 컬럼 선택(_col) -> 'Adj Close' 기본값
1. _df의 복사본을 생성 (df 변수 생성) -> 원본 데이터를 유지하기 위해
2. 컬럼 중에 Date가 존재한다면 Date컬럼을 인덱스로 변경
3. 인덱스를 시계열 데이터로 변경 
4. 결측치와 양의 무한대, 음의 무한대 데이터 제거 
4. _start와 _end를 기준으로 인덱스 필터링을 하고 _col을 기준으로 컬럼의 필터링을 해준다. (시도하고 문제가 발생한다면 인자값이 잘못되었다 출력하고 함수 종료)
5. 일별 수익율(rtn)을 생성하여 pct_change() + 1  데이터를 대입 
6. 누적 수익율을 계산하여 새로운 컬럼 (acc_rtn)에 대입  
7. 만들어진 데이터프레임과 최종 누적수익율을 되돌려준다. 


In [15]:
def bnh(
    _df, 
    _start = '2010-01-01', 
    _end = datetime.now(), 
    _col = 'Adj Close'
):
    # 데이터프레임의 복사본 생성 
    df = _df.copy()
    # try:
    #     _start = datetime.strptime(_start, '%Y-%m-%d')
    #     # 만약에 _end의 타입이 문자라면?
    #     if type(_end) == 'str':
    #         _end = datetime.strptime(_end, '%Y-%m-%d')
    # except:
    #     print('시간의 포멧이 맞지 않습니다. (YYYY-mm-dd)')
    #     return ""
    # Date 가 컬럼에 존재하면 Date를 인덱스로 변경 
    if 'Date' in df.columns:
        df.set_index('Date', inplace = True)
    # 인덱스를 시계열 데이터로 변경 
    df.index = pd.to_datetime(df.index)
    # 결측치와 무한대 값 제거
    flag = df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    df = df.loc[~flag, ]
    try :
        df = df.loc[_start : _end, [_col]]
    except Exception as e:
        print(e)
        print('입력 된 인자값이 잘못되었습니다.')
        return ""
    # 일별 수익율 계산하여 rtn 컬럼에 대입
    df['rtn'] = (df[_col].pct_change() + 1).fillna(1)
    # 누적 수익율 계산하여 acc_rtn 컬럼에 대입 
    df['acc_rtn'] = df['rtn'].cumprod()
    acc_rtn = df.iloc[-1, -1]
    # 결과 데이터프레임과 최종 누적수익율을 되돌려준다. 
    return df, acc_rtn

In [16]:
test_df = pd.read_csv("../../csv/AAPL.csv")

In [17]:
bnh(test_df, _start = '2010/01/01')

(             Adj Close       rtn   acc_rtn
 Date                                      
 2010-01-04   26.782711  1.000000  1.000000
 2010-01-05   26.829010  1.001729  1.001729
 2010-01-06   26.402260  0.984094  0.985795
 2010-01-07   26.353460  0.998152  0.983973
 2010-01-08   26.528664  1.006648  0.990515
 ...                ...       ...       ...
 2019-06-18  198.449997  1.023518  7.409631
 2019-06-19  197.869995  0.997077  7.387975
 2019-06-20  199.460007  1.008036  7.447342
 2019-06-21  198.779999  0.996591  7.421952
 2019-06-24  199.169998  1.001962  7.436514
 
 [2384 rows x 3 columns],
 7.436513727083075)